In [103]:
import numpy as np
import pandas as pd
import torch
from torch import device
from torch import nn
import torchvision.models as models
from torchvision.models import alexnet
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
# !pip install torch-summary
from torchsummary import summary
from torchvision import transforms
# !pip install imutils
from imutils import paths
import cv2
import os

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [104]:
device = device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


AlexNet

In [105]:
alexnet = alexnet(weights='AlexNet_Weights.DEFAULT').to(device)
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

Description of Layers of AlexNet

In [106]:
summary(alexnet)

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─AdaptiveAvgPool2d: 1-2                 --
├─Sequential: 1-3                        --
|    └─Dropout: 2-14                     --
|    └─Linear: 2-15                      37,752,832
|    └─ReLU: 2-16                        --
|    └─Dropout: 2-17                     --
|    └─Linear: 2-18                    

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─AdaptiveAvgPool2d: 1-2                 --
├─Sequential: 1-3                        --
|    └─Dropout: 2-14                     --
|    └─Linear: 2-15                      37,752,832
|    └─ReLU: 2-16                        --
|    └─Dropout: 2-17                     --
|    └─Linear: 2-18                    

Removing the fully connected layers to create feature extractor

In [107]:
alexnet = nn.Sequential(*list(alexnet.children())[:-1])
alexnet.eval()
# Check the modified GoogLeNet architecture
summary(alexnet)

# Now you can use this modified GoogLeNet as a feature extractor

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─AdaptiveAvgPool2d: 1-2                 --
Total params: 2,469,696
Trainable params: 2,469,696
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─AdaptiveAvgPool2d: 1-2                 --
Total params: 2,469,696
Trainable params: 2,469,696
Non-trainable params: 0

Transform function for images

In [108]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

In [109]:
print(transform)

Compose(
    ToPILImage()
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


Feature extraction from an image

In [110]:
def getFeatures(imageList):
    features_array = []
    for i in range(len(imageList)):
#         if i == 1165:
#             continue
        img = cv2.imread(imageList[i])
    #     print(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = transform(img)
        img = img.reshape(1, 3, 224, 224)
        img = img.to(device)
        with torch.no_grad():
            img_features = alexnet(img)
        img_features = img_features.cpu().detach().numpy().reshape(-1)
        features_array.append(img_features)
    return features_array

Function to load Dataset

In [111]:
def loadDataset(datasetPath):
    imageList = []

    directories = [f for f in os.listdir(datasetPath) if os.path.isdir(os.path.join(datasetPath, f))]

    labelList = []
    label = -1

    for directory in directories:
        label = str(directory)
        # path = resized+'/'+directory
        path = os.path.join(datasetPath, directory)

        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

        for file in files:
            imagePath = path+'/'+file
            imageList.append(imagePath)
            labelList.append(label)

    return (imageList,labelList)

Getting the classes of the dataset

In [112]:
datasetPath = 'flower_data/train'
# datasetPath = '/kaggle/input/bikevshorse'

directories = [f for f in os.listdir(datasetPath) if os.path.isdir(os.path.join(datasetPath, f))]
directories = [int(element) for element in directories]
print(sorted(directories))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102]


In [113]:
imageList,labelList = loadDataset(datasetPath)
print(labelList)

['67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '67', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '49', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95

In [114]:
features_array = getFeatures(imageList)

In [115]:
print(len(features_array))

X_train = np.array(features_array)
copyLabelList = labelList
# del copyLabelList[1165]
Y_train = copyLabelList

len(Y_train)

6552


6552

In [116]:
import random
c = list(zip(X_train, Y_train))

random.shuffle(c)

X_train, Y_train = zip(*c)

In [117]:
datasetPath = 'flower_data/valid'
# datasetPath = '/kaggle/input/bikevshorse'

directories = [f for f in os.listdir(datasetPath) if os.path.isdir(os.path.join(datasetPath, f))]
directories = [int(element) for element in directories]
print(sorted(directories))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102]


In [118]:
imageList,labelList = loadDataset(datasetPath)
print(labelList)

['67', '67', '49', '49', '49', '49', '49', '49', '49', '49', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '95', '78', '78', '78', '78', '78', '78', '78', '78', '78', '78', '78', '99', '99', '99', '99', '99', '99', '15', '15', '15', '15', '15', '15', '15', '82', '82', '82', '82', '82', '82', '82', '82', '82', '82', '82', '82', '82', '58', '58', '58', '58', '58', '58', '58', '58', '58', '58', '58', '58', '58', '58', '80', '80', '80', '80', '80', '80', '80', '80', '80', '80', '80', '80', '68', '68', '68', '68', '68', '68', '68', '68', '98', '98', '98', '98', '98', '98', '98', '98', '98', '98', '69', '69', '69', '69', '69', '19', '19', '19', '19', '66', '66', '66', '66', '66', '66', '27', '26', '26', '26', '65', '65', '65', '65', '65', '65', '65', '24', '24', '24', '24', '24', '35', '35', '35', '35', '102', '102', '102', '102', '102', '102', '94', '94', '94', '94', '94', '94', '94', '94', '94', '94', '94', '94', '94', '94', '85', '85', '85', '85', '85', '96', '96

In [119]:
features_array = getFeatures(imageList)

Extraction of features for checking accuracy on validation data

In [120]:
print(len(features_array))

X_test = np.array(features_array)
copyLabelList = labelList
# del copyLabelList[1165]
Y_test = copyLabelList

# len(Y_train)

818


Random Forest with Max depth = 20

In [121]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=42)
clf.fit(X_train, Y_train)
clf.score(X_test,Y_test)

0.6528117359413202

Logistic Regression with max 100 iterations

In [122]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=100).fit(X_train, Y_train)
y_pred = clf.predict(X_test)
clf.score(X_test,Y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.91320293398533

KNN with N = 5

In [123]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, Y_train)
neigh.score(X_test,Y_test)

0.589242053789731

SVM with linear Kernel

In [124]:
# Train a classifier using the flattened features
classifier = SVC(kernel='linear')
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

In [125]:
# Make predictions on the test set using the flattened features
predictions = classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.902200488997555


Random Forest with no constraints

In [126]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, Y_train)
clf.score(X_test,Y_test)

0.6687041564792175

### Bike vs Horse Classification

In [127]:
image_paths = 'Assignment2_BikeHorses'
# image_paths = list(paths.list_images('/kaggle/input/bikevshorse'))
images = []
labels = []

In [128]:
def loadDataset(datasetPath):
    imageList = []

    directories = [f for f in os.listdir(datasetPath) if os.path.isdir(os.path.join(datasetPath, f))]

    labelList = []
    label = -1

    for directory in directories:
        label = str(directory)
        # path = resized+'/'+directory
        path = os.path.join(datasetPath, directory)

        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

        for file in files:
            imagePath = path+'/'+file
            imageList.append(imagePath)
            labelList.append(label)

    return (imageList,labelList)

In [129]:
imageList,labelList = loadDataset(image_paths)
print(imageList)
print(labelList)

['Assignment2_BikeHorses/Bikes/0007.jpg', 'Assignment2_BikeHorses/Bikes/0069.jpg', 'Assignment2_BikeHorses/Bikes/0059.jpg', 'Assignment2_BikeHorses/Bikes/0053.jpg', 'Assignment2_BikeHorses/Bikes/0003.jpg', 'Assignment2_BikeHorses/Bikes/0070.jpg', 'Assignment2_BikeHorses/Bikes/0004.jpg', 'Assignment2_BikeHorses/Bikes/0072.jpg', 'Assignment2_BikeHorses/Bikes/0049.jpg', 'Assignment2_BikeHorses/Bikes/0006.jpg', 'Assignment2_BikeHorses/Bikes/0054.jpg', 'Assignment2_BikeHorses/Bikes/0066.jpg', 'Assignment2_BikeHorses/Bikes/0051.jpg', 'Assignment2_BikeHorses/Bikes/0033.jpg', 'Assignment2_BikeHorses/Bikes/0021.jpg', 'Assignment2_BikeHorses/Bikes/0071.jpg', 'Assignment2_BikeHorses/Bikes/0017.jpg', 'Assignment2_BikeHorses/Bikes/0010.jpg', 'Assignment2_BikeHorses/Bikes/0005.jpg', 'Assignment2_BikeHorses/Bikes/0045.jpg', 'Assignment2_BikeHorses/Bikes/0025.jpg', 'Assignment2_BikeHorses/Bikes/0013.jpg', 'Assignment2_BikeHorses/Bikes/0074.jpg', 'Assignment2_BikeHorses/Bikes/0011.jpg', 'Assignment2_Bi

In [131]:
features_array = getFeatures(imageList)

In [132]:
print(len(features_array))

179


In [133]:
X_train = np.array(features_array)
Y_train = np.array(labelList)
print(X_train.shape)
print(Y_train.shape)

(179, 9216)
(179,)


Train Test Split

In [134]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)

X_train.shape

(125, 9216)

Scaling Training Data

In [135]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_val)

array([[-0.31784603, -0.2773541 , -0.2186023 , ..., -0.25831264,
        -0.21680301, -0.3373043 ],
       [-0.31784603, -0.2773541 , -0.2186023 , ..., -0.25831264,
        -0.21680301,  2.0610776 ],
       [-0.31784603, -0.2773541 , -0.2186023 , ..., -0.25831264,
        -0.21680301, -0.3373043 ],
       ...,
       [-0.31784603, -0.2773541 , -0.2186023 , ..., -0.25831264,
        -0.21680301, -0.3373043 ],
       [-0.31784603, -0.2773541 , -0.2186023 , ..., -0.25831264,
        -0.21680301, -0.3373043 ],
       [-0.31784603, -0.2773541 , -0.2186023 , ..., -0.25831264,
        -0.21680301,  2.773259  ]], dtype=float32)

Logisic 

Logistic Regression

In [136]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=100).fit(X_train, Y_train)
y_pred = clf.predict(X_val)
clf.score(X_val,Y_val)

1.0

Support Vector Machine

In [137]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

clf = SVC(gamma='auto',kernel='linear')
clf.fit(X_train, Y_train)
clf.score(X_val,Y_val)

1.0

Random Forest with Max Depth 20

In [138]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=42)
clf.fit(X_train, Y_train)
clf.score(X_val,Y_val)

1.0

Random Forest with Max Depth 60

In [139]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=60, random_state=42)
clf.fit(X_train, Y_train)
clf.score(X_val,Y_val)

1.0

KNN with N = 5

In [140]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, Y_train)
neigh.score(X_val,Y_val)

1.0